In [1]:
import tensorflow as tf
import numpy as np
import pickle
import time
import os
import datetime
from tensorflow.contrib import learn
import data_helpers
from text_cnn import TextCNN

# Parameters

# Data loading Parameters
tf.flags.DEFINE_float('dev_sample_percetage', 0.1, 'Percetafe of training data to use for vaildation')
tf.flags.DEFINE_string('data_file', r'F:\post graduate\dataset\train1.csv', 'Data Source')

# Model Hyperparams
tf.flags.DEFINE_integer('embedding', 100, 'Dimension of embedding')
tf.flags.DEFINE_string('filter_sizes', '3,4,5', 'Filter size')
tf.flags.DEFINE_integer('num_filters', 128, 'Number of filters')
tf.flags.DEFINE_float('dropout_keep_prob', 0.5, 'Dropout')
tf.flags.DEFINE_float('L2_reg_Lamda', 0.001, 'L2')

# Training Parameters
tf.flags.DEFINE_integer('batch_size', 128, 'Batch size')
tf.flags.DEFINE_integer('num_epochs', 25, 'Number of epoch')
tf.flags.DEFINE_integer('evaluate_every', 100, 'Evaluate_every')
tf.flags.DEFINE_integer('checkpoint_every', 100, 'Saving...')
tf.flags.DEFINE_integer('num_checkpoints', 5, 'Number of checkpoints to store')

tf.flags.DEFINE_boolean('log_device_placement', False, 'Log placement of ops on devices')
tf.flags.DEFINE_boolean('allow_soft_placement', True, 'Allow device soft device placement')
# Misc Parameters
FLAGS = tf.flags.FLAGS  # FLAGS保存命令行参数的数据
FLAGS.flag_values_dict()  # 将其解析成字典存储到FLAGS.__flags中
print('Patameters')
for k in FLAGS:
    print(k.upper(), FLAGS[k].value)


def prepocess():
    # loda data
    print("Loading data...")
    x_text, y = data_helpers.load_data_and_labels(FLAGS.data_file)

    # bulid vocabulary
    max_document_length = max(len(x.split(' ')) for x in x_text)
    vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length)
    with open(r'F:\bug_triage\emb_and_dict\dict100.pickle', 'rb') as f:
        vocab = pickle.load(f)
    vocab_processor.fit(vocab)
    x = np.array(list(vocab_processor.transform(x_text)))

    # shuffle
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = x[shuffle_indices]
    y_shuffled = y[shuffle_indices]

    # split train/test dataset
    dev_sample_index = -1 * int(FLAGS.dev_sample_percetage * float(len(y)))
    x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
    y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

    del x, y, x_shuffled, y_shuffled

    print('Vocabulary Size: {:d}'.format(len(vocab_processor.vocabulary_)))
    print('Train/Dev split: {:d}/{:d}'.format(len(y_train), len(y_dev)))
    return x_train, y_train, vocab_processor, x_dev, y_dev


def train(x_train, y_train, vocab_processor, x_dev, y_dev):
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
            allow_soft_placement=FLAGS.allow_soft_placement,
            log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=FLAGS.embedding,
                filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                num_filters=FLAGS.num_filters,
                l2_reg_lambda=FLAGS.L2_reg_Lamda
            )

        # Define Training procedure
        global_step = tf.Variable(0, name='global_step', trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        # minimize()的第一步，返回(gradient, variable)对的list
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        # minimize()的第二部分，返回一个执行梯度更新的ops
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
#         if not os.path.exists(checkpoint_dir):
#             os.makedirs(checkpoint_dir)
#         saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

#         # Write vocabulary
#         vocab_processor.save(os.path.join(out_dir, "vocab"))

        # 初始化所有变量
        sess.run(tf.global_variables_initializer())

    def train_step(x_batch, y_batch):
        feed_dict = {
            cnn.input_x: x_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: FLAGS.dropout_keep_prob}
        _, step, summaries, loss, accuracy = sess.run(
            [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
            feed_dict)
        time_str = datetime.datetime.now().isoformat()
        print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
        train_summary_writer.add_summary(summaries, step)

    def dev_step(x_batch, y_batch, writer=None):
        feed_dict = {
            cnn.input_x: x_batch,
            cnn.input_y: y_batch,
            cnn.dropout_keep_prob: 1.0
        }
        step, summaries, loss, accuracy = sess.run(
            [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
            feed_dict=feed_dict
        )
        time_str = datetime.datetime.now()
        print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
#         if writer:
#             writer.add_summary(summaries, step)

    # generate batches
    batches = data_helpers.batch_iter(list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

    # training loop
    for batch in batches:
        x_batch, y_batch = zip(*batch)
        train_step(x_batch, y_batch)
        current_step = tf.train.global_step(sess, global_step)
        if current_step % FLAGS.evaluate_every == 0:
            print('\n Evaluation:')
            dev_step(x_dev, y_dev, writer=dev_summary_writer)
#         if current_step % FLAGS.checkpoint_every == 0:
#             path = saver.save(sess, checkpoint_prefix, global_step=current_step)  # 指定保存路径
#             print('Save model checkpoint to {} \n'.format(path))


def main(argv=None):
    x_train, y_train, vocab_processor, x_dev, y_dev = prepocess()
    train(x_train, y_train, vocab_processor, x_dev, y_dev)


if __name__ == '__main__':
    tf.app.run()


Patameters
DEV_SAMPLE_PERCETAGE 0.1
DATA_FILE F:\post graduate\dataset\train1.csv
EMBEDDING 100
FILTER_SIZES 3,4,5
NUM_FILTERS 128
DROPOUT_KEEP_PROB 0.5
L2_REG_LAMDA 0.001
BATCH_SIZE 128
NUM_EPOCHS 25
EVALUATE_EVERY 100
CHECKPOINT_EVERY 100
NUM_CHECKPOINTS 5
LOG_DEVICE_PLACEMENT False
ALLOW_SOFT_PLACEMENT True
Loading data...
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Vocabulary Size: 370847
Train/Dev split: 43839/4871
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sp

2019-07-02T16:23:16.130854: step 72, loss 2.77788, acc 0.25
2019-07-02T16:23:16.175735: step 73, loss 3.14721, acc 0.234375
2019-07-02T16:23:16.217622: step 74, loss 2.94117, acc 0.25
2019-07-02T16:23:16.260508: step 75, loss 2.81879, acc 0.257812
2019-07-02T16:23:16.302396: step 76, loss 2.88138, acc 0.257812
2019-07-02T16:23:16.346279: step 77, loss 2.92309, acc 0.234375
2019-07-02T16:23:16.390161: step 78, loss 2.85689, acc 0.242188
2019-07-02T16:23:16.432049: step 79, loss 2.79451, acc 0.265625
2019-07-02T16:23:16.472940: step 80, loss 2.7503, acc 0.257812
2019-07-02T16:23:16.514828: step 81, loss 2.73873, acc 0.289062
2019-07-02T16:23:16.556716: step 82, loss 2.79842, acc 0.289062
2019-07-02T16:23:16.600599: step 83, loss 3.05348, acc 0.257812
2019-07-02T16:23:16.641489: step 84, loss 2.96182, acc 0.273438
2019-07-02T16:23:16.682379: step 85, loss 2.77601, acc 0.257812
2019-07-02T16:23:16.723271: step 86, loss 2.78812, acc 0.234375
2019-07-02T16:23:16.766156: step 87, loss 2.67728

2019-07-02T16:23:22.439989: step 200, loss 1.75891, acc 0.460938

 Evaluation:
2019-07-02 16:23:22.903747: step 200, loss 1.48428, acc 0.525765
2019-07-02T16:23:22.936658: step 201, loss 1.88546, acc 0.4375
2019-07-02T16:23:22.977549: step 202, loss 1.89656, acc 0.398438
2019-07-02T16:23:23.018439: step 203, loss 1.68992, acc 0.507812
2019-07-02T16:23:23.059330: step 204, loss 1.77983, acc 0.445312
2019-07-02T16:23:23.100221: step 205, loss 1.86099, acc 0.40625
2019-07-02T16:23:23.145101: step 206, loss 1.74399, acc 0.46875
2019-07-02T16:23:23.186989: step 207, loss 1.78637, acc 0.414062
2019-07-02T16:23:23.227879: step 208, loss 1.70303, acc 0.460938
2019-07-02T16:23:23.269768: step 209, loss 1.93251, acc 0.429688
2019-07-02T16:23:23.313650: step 210, loss 1.83962, acc 0.421875
2019-07-02T16:23:23.354541: step 211, loss 1.6837, acc 0.445312
2019-07-02T16:23:23.395432: step 212, loss 1.84845, acc 0.46875
2019-07-02T16:23:23.436322: step 213, loss 1.84823, acc 0.421875
2019-07-02T16:23:

2019-07-02T16:23:28.659357: step 325, loss 1.55732, acc 0.476562
2019-07-02T16:23:28.703240: step 326, loss 1.36613, acc 0.570312
2019-07-02T16:23:28.744131: step 327, loss 1.59832, acc 0.453125
2019-07-02T16:23:28.785022: step 328, loss 1.60172, acc 0.460938
2019-07-02T16:23:28.825912: step 329, loss 1.49119, acc 0.546875
2019-07-02T16:23:28.867800: step 330, loss 1.44549, acc 0.539062
2019-07-02T16:23:28.912680: step 331, loss 1.4431, acc 0.507812
2019-07-02T16:23:28.954569: step 332, loss 1.63975, acc 0.484375
2019-07-02T16:23:28.994461: step 333, loss 1.42361, acc 0.59375
2019-07-02T16:23:29.036350: step 334, loss 1.60785, acc 0.5625
2019-07-02T16:23:29.077240: step 335, loss 1.49426, acc 0.5
2019-07-02T16:23:29.122120: step 336, loss 1.40989, acc 0.492188
2019-07-02T16:23:29.164008: step 337, loss 1.64549, acc 0.492188
2019-07-02T16:23:29.204899: step 338, loss 1.57507, acc 0.507812
2019-07-02T16:23:29.246787: step 339, loss 1.55305, acc 0.414062
2019-07-02T16:23:29.288675: step 3

2019-07-02T16:23:34.679263: step 456, loss 1.37608, acc 0.492188
2019-07-02T16:23:34.726137: step 457, loss 1.75748, acc 0.421875
2019-07-02T16:23:34.773012: step 458, loss 1.28883, acc 0.5625
2019-07-02T16:23:34.814900: step 459, loss 1.65632, acc 0.4375
2019-07-02T16:23:34.856788: step 460, loss 1.31643, acc 0.578125
2019-07-02T16:23:34.897679: step 461, loss 1.49368, acc 0.523438
2019-07-02T16:23:34.944553: step 462, loss 1.35372, acc 0.53125
2019-07-02T16:23:34.985444: step 463, loss 1.31277, acc 0.5
2019-07-02T16:23:35.027332: step 464, loss 1.24775, acc 0.546875
2019-07-02T16:23:35.069220: step 465, loss 1.45289, acc 0.476562
2019-07-02T16:23:35.114100: step 466, loss 1.39374, acc 0.523438
2019-07-02T16:23:35.154991: step 467, loss 1.50201, acc 0.5
2019-07-02T16:23:35.196879: step 468, loss 1.34601, acc 0.539062
2019-07-02T16:23:35.237769: step 469, loss 1.33252, acc 0.539062
2019-07-02T16:23:35.279658: step 470, loss 1.37868, acc 0.539062
2019-07-02T16:23:35.320548: step 471, lo

...

2019-07-02T16:29:55.898023: step 8305, loss 0.600237, acc 0.835938
2019-07-02T16:29:55.940908: step 8306, loss 0.790031, acc 0.765625
2019-07-02T16:29:55.981799: step 8307, loss 0.704384, acc 0.78125
2019-07-02T16:29:56.023687: step 8308, loss 0.624609, acc 0.804688
2019-07-02T16:29:56.064577: step 8309, loss 0.633629, acc 0.820312
2019-07-02T16:29:56.106466: step 8310, loss 0.696601, acc 0.773438
2019-07-02T16:29:56.148353: step 8311, loss 0.663751, acc 0.796875
2019-07-02T16:29:56.189244: step 8312, loss 0.609944, acc 0.828125
2019-07-02T16:29:56.231131: step 8313, loss 0.618549, acc 0.765625
2019-07-02T16:29:56.275015: step 8314, loss 0.662214, acc 0.765625
2019-07-02T16:29:56.317900: step 8315, loss 0.602069, acc 0.789062
2019-07-02T16:29:56.360786: step 8316, loss 0.580819, acc 0.804688
2019-07-02T16:29:56.401676: step 8317, loss 0.649886, acc 0.789062
2019-07-02T16:29:56.444561: step 8318, loss 0.601445, acc 0.835938
2019-07-02T16:29:56.487446: step 8319, loss 0.60173, acc 0.8046

2019-07-02T16:30:02.056558: step 8430, loss 0.880693, acc 0.726562
2019-07-02T16:30:02.099443: step 8431, loss 0.768039, acc 0.742188
2019-07-02T16:30:02.141330: step 8432, loss 0.715479, acc 0.742188
2019-07-02T16:30:02.183218: step 8433, loss 0.742116, acc 0.742188
2019-07-02T16:30:02.226103: step 8434, loss 0.532691, acc 0.835938
2019-07-02T16:30:02.271981: step 8435, loss 0.705705, acc 0.742188
2019-07-02T16:30:02.313869: step 8436, loss 0.746632, acc 0.8125
2019-07-02T16:30:02.354760: step 8437, loss 0.571412, acc 0.820312
2019-07-02T16:30:02.397647: step 8438, loss 0.657311, acc 0.773438
2019-07-02T16:30:02.446515: step 8439, loss 0.67988, acc 0.765625
2019-07-02T16:30:02.498376: step 8440, loss 0.706491, acc 0.757812
2019-07-02T16:30:02.544253: step 8441, loss 0.685205, acc 0.757812
2019-07-02T16:30:02.606093: step 8442, loss 0.800452, acc 0.75
2019-07-02T16:30:02.654957: step 8443, loss 0.773884, acc 0.773438
2019-07-02T16:30:02.696845: step 8444, loss 0.675435, acc 0.789062
20

2019-07-02T16:30:08.137299: step 8555, loss 0.627446, acc 0.820312
2019-07-02T16:30:08.179187: step 8556, loss 0.621352, acc 0.820312
2019-07-02T16:30:08.222073: step 8557, loss 0.798828, acc 0.75
2019-07-02T16:30:08.265955: step 8558, loss 0.55695, acc 0.804688
2019-07-02T16:30:08.307843: step 8559, loss 0.809412, acc 0.710938
2019-07-02T16:30:08.351726: step 8560, loss 0.622757, acc 0.851562
2019-07-02T16:30:08.394611: step 8561, loss 0.788199, acc 0.78125
2019-07-02T16:30:08.435502: step 8562, loss 0.797309, acc 0.757812
2019-07-02T16:30:08.477390: step 8563, loss 0.670658, acc 0.789062
2019-07-02T16:30:08.519278: step 8564, loss 0.680152, acc 0.78125
2019-07-02T16:30:08.562163: step 8565, loss 0.801167, acc 0.742188
2019-07-02T16:30:08.606046: step 8566, loss 0.876755, acc 0.71875
2019-07-02T16:30:08.648931: step 8567, loss 0.624265, acc 0.78125
2019-07-02T16:30:08.689822: step 8568, loss 0.792504, acc 0.710938
2019-07-02T16:30:08.732707: step 8569, loss 0.755407, acc 0.734375
2019

SystemExit: 

C:\Users\Snake\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
